In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("2019.csv", sep="|")

In [9]:
data.head()

,ValidDate,ValidTime,AQSID,SiteName,GMTOffset,MeasurementType,MeasurementUnit,RawMeasurement,DataSource
0,01/01/19,00:00,250250002,BOSTON-KENMORE,-5,NO,PPB,3.0,Massachusetts Dept. of Environmental Protection
1,01/01/19,00:00,250250002,BOSTON-KENMORE,-5,NO2,PPB,11.0,Massachusetts Dept. of Environmental Protection
2,01/01/19,00:00,250250002,BOSTON-KENMORE,-5,NOX,PPB,15.0,Massachusetts Dept. of Environmental Protection
3,01/01/19,00:00,250250002,BOSTON-KENMORE,-5,SO2,PPB,0.0,Massachusetts Dept. of Environmental Protection
4,01/01/19,00:00,250250042,Boston - Roxbury,-5,BARPR,MILLIBAR,1021.1,Massachusetts Dept. of Environmental Protection


In [10]:
data = data.drop(['GMTOffset', 'ValidTime', 'DataSource'], axis=1)

data.head()

,ValidDate,AQSID,SiteName,MeasurementType,MeasurementUnit,RawMeasurement
0,01/01/19,250250002,BOSTON-KENMORE,NO,PPB,3.0
1,01/01/19,250250002,BOSTON-KENMORE,NO2,PPB,11.0
2,01/01/19,250250002,BOSTON-KENMORE,NOX,PPB,15.0
3,01/01/19,250250002,BOSTON-KENMORE,SO2,PPB,0.0
4,01/01/19,250250042,Boston - Roxbury,BARPR,MILLIBAR,1021.1


In [11]:
data = data[(data['MeasurementType'] == 'NO2') | (data['MeasurementType'] == 'PM2.5') | (data['MeasurementType'] == 'OZONE') ]

data.head()

,ValidDate,AQSID,SiteName,MeasurementType,MeasurementUnit,RawMeasurement
1,01/01/19,250250002,BOSTON-KENMORE,NO2,PPB,11.0
7,01/01/19,250250042,Boston - Roxbury,NO2,PPB,9.0
10,01/01/19,250250042,Boston - Roxbury,OZONE,PPB,23.0
11,01/01/19,250250042,Boston - Roxbury,PM2.5,UG/M3,7.0
23,01/01/19,250250044,Boston-Von Hillern,NO2,PPB,8.0


In [12]:
data[data["MeasurementType"] == "PM2.5"].value_counts()

ValidDate  AQSID      SiteName            MeasurementType  MeasurementUnit  RawMeasurement
08/26/19   250250044  Boston-Von Hillern  PM2.5            UG/M3            6.0               11
11/28/19   250250042  Boston - Roxbury    PM2.5            UG/M3            6.0               10
10/12/19   250250044  Boston-Von Hillern  PM2.5            UG/M3            4.0               10
05/05/19   250250044  Boston-Von Hillern  PM2.5            UG/M3            2.0               10
10/29/19   250250042  Boston - Roxbury    PM2.5            UG/M3            6.0               10
                                                                                              ..
06/23/19   250250044  Boston-Von Hillern  PM2.5            UG/M3            8.0                1
                                                                            10.0               1
06/24/19   250250042  Boston - Roxbury    PM2.5            UG/M3            4.0                1
                                    

In [14]:
import numpy as np
import aqi 

groups = data.groupby(['ValidDate', 'SiteName', 'AQSID']).indices
new_dict = {
    "ValidDate": [],
    "AQSID": [],
    "SiteName": [],
    "OZONE_AQI": [],
    "PM25_AQI": [],
    "NO2_AQI": [],
}

for (date, site, id), indices in groups.items():
    new_dict["ValidDate"].append(date)
    new_dict["SiteName"].append(site)
    new_dict["AQSID"].append(id)

    ozone_aqi = np.nan
    pm25_aqi = np.nan
    no2_aqi = np.nan

    for i in indices:
        row = data.iloc[i]

        try:
            if row['MeasurementType'] == "NO2":
                if row['MeasurementUnit'] == "PPB":
                    no2_aqi = aqi.to_iaqi(aqi.POLLUTANT_NO2_1H, str(row['RawMeasurement']), algo=aqi.ALGO_EPA)
                else:
                    ppb = row['RawMeasurement'] * 0.51 # Convert from ug/m3 to ppb
                    no2_aqi = aqi.to_iaqi(aqi.POLLUTANT_NO2_1H, str(ppb), algo=aqi.ALGO_EPA)

            elif row['MeasurementType'] == "OZONE":
                if row['MeasurementUnit'] == "PPB":
                    ppm = row['RawMeasurement'] * 0.001 # Convert from ppb to ppm
                    ozone_aqi = aqi.to_iaqi(aqi.POLLUTANT_O3_8H, str(ppm), algo=aqi.ALGO_EPA)
                else:
                    ppm = row['RawMeasurement'] * 0.001 * 0.49 # Convert from ug/m3 to ppm
                    ozone_aqi = aqi.to_iaqi(aqi.POLLUTANT_O3_8H, str(ppm), algo=aqi.ALGO_EPA)

            elif row['MeasurementType'] == "PM2.5":
                if row['MeasurementUnit'] == "UG/M3":
                    no2_aqi = aqi.to_iaqi(aqi.POLLUTANT_PM25, str(row['RawMeasurement']), algo=aqi.ALGO_EPA)
        except:
            print(i)
    
    new_dict["OZONE_AQI"].append(ozone_aqi)
    new_dict["PM25_AQI"].append(pm25_aqi)
    new_dict["NO2_AQI"].append(no2_aqi)


new_data = pd.DataFrame(new_dict)
new_data.head(10)

ModuleNotFoundError: No module named 'aqi'